In [ ]:
"""
Created By    : Jared W. Marquis
Creation Date : 01 August 2022
Course        : ATSC 528 - Atmospheric Data Analysis
Assignment    : #03 - Statistical Objective Analysis

Purpose:
Script to take sparse upper air observations and analyze them on a
polar stereographic map projection using statistical objective analysis.
[PUT MORE INFORMATION HERE - I.E., WHAT SPECIFIC THING IS BEING DONE]

"""
__author__    = "Jared W. Marquis"
__contact__   = "jared.marquis@und.edu"

In [ ]:
#Used Colab for further works, supports Pip
!pip install cartopy

In [ ]:
### Import Required Modules (shouldn't need to change) ###
import numpy as np                 #numpy for math
import matplotlib.pyplot as plt    #matplotlib for plotting
import cartopy.crs as ccrs         #cartopy for plotting on map
import cartopy.feature as cfeature #cartopy basic shapefiles

In [ ]:
### Read in observations ###
data_path = '/content/RAOBs_201903131200.txt'
data = np.genfromtxt(data_path, delimiter=',', dtype=str)

# Individual columns for each data
station_ids = data[:, 0]  # Station IDs
station_lat = data[:, 1].astype(float)  # Latitude values
station_lon = data[:, 2].astype(float)  # Longitude values
heights = data[:, 3].astype(float)  # 500-mb Height


In [ ]:
phi0 = np.radians(60)  # central latitude φ0 in radians
lambda0 = np.radians(-115)
lat_rad=np.radians (station_lat)
lon_rad=np.radians(station_lon) # central longitude λ0 in radians
R = 6371*1000  # Earth's radius in meters
m = 1 / 15000000  # map scale

proj= ccrs.Stereographic(central_latitude=90, central_longitude=-115,true_scale_latitude=60)

In [ ]:
### Set up analysis map with a 22x28 rectangular grid of points ###
x0, y0 = 18.90, -6.30
dx, dy = 1.27, 1.27
nx, ny = 22,28
# Making a (22x28) grid
x_grid = np.linspace(x0, x0 + (nx - 1) * dx, nx)
y_grid = np.linspace(y0, y0 + (ny - 1) * dy, ny)

X, Y = np.meshgrid(x_grid, y_grid)
print("Shape of x_grid:", x_grid.shape)
print("Shape of y_grid:", y_grid.shape)
plt.scatter(X, Y, color='blue', label='Grid Points', marker='.')

In [ ]:
# Function to convert latitude and longitude to x, y in stereographic projection
def latlon_to_xy(station_lat, station_lon):

  lat_rad = np.radians(station_lat)
  lon_rad = np.radians(station_lon)

  sigma = (1+np.sin(phi0)) / (1+np.sin(lat_rad))

  x_obs = R * sigma * m *np.cos(lat_rad)*np.cos(np.radians(station_lon)-lambda0)*100

  y_obs = R * sigma * m *np.cos(lat_rad)*np.sin(np.radians(station_lon)-lambda0)*100



  return x_obs, y_obs

# Apply the function to all latitude and longitude values
obs_coords = np.array([latlon_to_xy(lat, lon) for lat, lon in zip(station_lat, station_lon)])

# Separate x and y coordinates
xk, yk = obs_coords[:, 0], obs_coords[:, 1]
# Check the shape of obs_coords
print(f"obs_coords shape: {obs_coords.shape}")

#Plot the Stations and the Grid to Verify Alignment (I got wrong here somewhere)
plt.scatter(X, Y, color='blue', label='Grid Points', marker='.')  # Plot the grid points
plt.scatter(xk, yk, color='green', label='Converted Station Positions', marker='o')  # Plot station positions

In [ ]:
r_e=12.7775892;
Kd=10.8844524

In [ ]:
b = 1.24  # Range factor (Schlatter)
b_prime = b * m**2  # Adjusted range factor for the grid

# Schlatter correlation model
def schlatter_correlation(s_squared, b_prime):
    # s_squared is the squared distance
    if s_squared == 0:
        return 1.0  # Catch the case where s = 0 to ensure ρ(0) = 1
    return 0.95 * np.exp(-b_prime * s_squared)

In [ ]:
def calculate_correlation_matrix(obs_coords, grid_coords, b_prime):
    # Compute distances between observation points (ρ_kl)
    distances_kl_squared = np.sum((obs_coords[:, None, :] - obs_coords[None, :, :])**2, axis=2)
    rho_kl = np.vectorize(schlatter_correlation)(distances_kl_squared, b_prime)

    # Compute distances between grid points and observation points (ρ_ki)
    distances_ki_squared = np.sum((grid_coords[:, None, :] - obs_coords[None, :, :])**2, axis=2)
    rho_ki = np.vectorize(schlatter_correlation)(distances_ki_squared, b_prime)

    return rho_kl, rho_ki

In [ ]:
weights = []
for grid_x, grid_y in zip(X.ravel(), Y.ravel()):
    # Compute distance between grid point and observation points
    distances_ki = np.sqrt((xk - grid_x)**2 + (yk - grid_y)**2)
    rho_ki = correlation_function(distances_ki**2, Kd)

    # Solve for weights: w_i = ρ^-1 * ρ_ki
    w_i = np.linalg.solve(rho_kl, rho_ki)
    weights.append(w_i)

In [ ]:
grid_coords = np.column_stack((X.ravel(), Y.ravel()))
rho_kl, rho_ki = calculate_correlation_matrix(obs_coords, grid_coords, b_prime)

# Precompute the inverse of the correlation matrix ρ_kl
rho_kl_inv = np.linalg.inv(rho_kl)

# Compute weights for all grid points
weights = rho_kl_inv @ rho_ki.T  # Matrix multiplication for all grid points

# Reshape weights for visualization
weights_grid = weights.T.reshape(X.shape + (len(xk),))  # Shape: (nx, ny, 135)

# Display one set of weights for a single grid point
weights_grid[0, 0]

In [ ]:
# Flatten the grid
grid_points = np.column_stack((x_grid.ravel(), y_grid.ravel()))
background_field = np.zeros(grid_points.shape[0])

for i, grid_point in enumerate(grid_points):
    # Compute distances to all observation points
    distances = cdist([grid_point], np.column_stack((obs_x, obs_y))).flatten()

    # Consider only observations within the radius of influence
    within_radius = distances < r_e
    weights = weights[within_radius]
    values = obs_height[within_radius]

    if weights.size > 0:
        background_field[i] = np.sum(weights * values) / np.sum(weights)
    else:
        background_field[i] = np.nan  # Mark as NaN if no points within radius

# Reshape the background field to grid shape
background_field = background_field.reshape(grid_shape)


In [ ]:
# Function to compute analysis field for N closest stations
def compute_analysis_field(N, X, Y, xk, yk, f_O, f_B_obs, f_B_grid):
    f_A_grid = np.zeros(X.shape)  # Initialize analysis field

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # Get the current grid point
            grid_x, grid_y = X[i, j], Y[i, j]

            # Compute distances from grid point to observation points
            distances = np.sqrt((xk - grid_x)**2 + (yk - grid_y)**2)

            # Get the indices of the N closest stations
            closest_indices = np.argsort(distances)[:N]

            # Extract weights, observation values, and background values for the closest stations
            rho_kl_inv_closest = np.linalg.inv(rho_kl[np.ix_(closest_indices, closest_indices)])
            rho_ki_closest = correlation_function(distances[closest_indices]**2, b_prime)
            weights_closest = rho_kl_inv_closest @ rho_ki_closest

            f_O_closest = f_O[closest_indices]
            f_B_obs_closest = f_B_obs[closest_indices]

            # Compute the analysis value
            f_A_grid[i, j] = f_B_grid[i, j] + np.sum(
                weights_closest * (f_O_closest - f_B_obs_closest)
            )

    return f_A_grid

In [ ]:
f_A_grid_N2 = compute_analysis_field(2, X, Y, xk, yk, f_O, f_B_obs, f_B_grid)
f_A_grid_N4 = compute_analysis_field(4, X, Y, xk, yk, f_O, f_B_obs, f_B_grid)
f_A_grid_N10 = compute_analysis_field(10, X, Y, xk, yk, f_O, f_B_obs, f_B_grid)

In [ ]:
X_meters = X * 0.01 # Convert cm to meters

Y_meters = Y * 0.01 # Convert cm to meters



# grid x-y coordinates back to latitude and longitude

x_squared_y_squared = (X_meters/m)**2 + (Y_meters/m)**2

lambda_grid = lambda0 + np.arctan(Y_meters/X_meters)

phi_grid = (np.pi/2) - 2*np.arctan((x_squared_y_squared**0.5)/(R*(1+np.sin(phi0))))



# Convert to degrees

latitude_grid = np.degrees(phi_grid)

longitude_grid = np.degrees(lambda_grid)

In [ ]:
### Plot 500mb analyses over a map ###
#convert x,y to lat/long

LAT = latitude_grid
LON = longitude_grid

ROI = [R1,R2,R3]

ANALYSIS=[Analysis1,Analysis2,Analysis3]

proj = ccrs.Stereographic(central_longitude=-115,central_latitude=90,true_scale_latitude=60)
fig = plt.figure(figsize=(5,15),dpi=200)
for i in range(len(ROI)):
    ax1 = fig.add_subplot(3,1,i+1,projection=proj)
    ax1.add_feature(cfeature.STATES)
    ax1.add_feature(cfeature.COASTLINE)
    cs1 = ax1.contour(LON,LAT,ANALYSIS[i],colors='k',levels=np.arange(0,8000,60),transform=ccrs.PlateCarree())
    plt.clabel(cs1,levels=np.arange(0,8000,60))
fig.tight_layout()
plt.show()


In [ ]:
def compute_rms_error(xk, yk, observations, analysis_grid, X, Y,RoI):

    grid_points = np.column_stack((X.flatten(), Y.flatten()))
    grid_values = analysis_grid.flatten()

    # Interpolate analysis values at observation points
    interpolated_analysis=bilinear_interpolation(xk, yk, X, Y, analysis_grid, RoI)


    # Calculate the squared differences
    differences = observations - interpolated_analysis
    squared_differences = differences ** 2

    # Compute RMS error
    rms_error = np.sqrt(np.nanmean(squared_differences))  # nanmean ignores any NaN values
    return rms_error
rmse1 = compute_rms_error(xk, yk, heights, Analysis1, X, Y,R1)
print("RMSE_1:", rmse1)
rmse2 = compute_rms_error(xk, yk, heights, Analysis2, X, Y,R2)
print("RMSE_2:", rmse2)
rmse3 = compute_rms_error(xk, yk, heights, Analysis3, X, Y,R3)
print("RMSE_3:", rmse3)


In [ ]:
### Store the number of observations available for each grid point in text files ###
np.savetxt("Analysis_first_pass.txt", Analysis1, fmt="%.6f", header="Analysis Heights for First Pass")
np.savetxt("Analysis_second_pass.txt", Analysis2, fmt="%.6f", header="Analysis Heights for Second Pass")
np.savetxt("Analysis_third_pass.txt", Analysis3, fmt="%.6f", header="Analysis Heights for Third Pass")

np.savetxt("Diff_2_1.txt", Analysis1, fmt="%.6f", header="Analysis2-Analysis1")
np.savetxt("Diff_3_1.txt", Analysis2, fmt="%.6f", header="Analysis3-Analysis1")
np.savetxt("Diff_3_2.txt", Analysis3, fmt="%.6f", header="Analysis3-Analysis2")

In [ ]:
### In a separte text file (or below), answer the following questions ###
'''
1 - Describe the general features that you see in your contoured analyses.


2 - Describe the differences that you see in your contoured analyses.
    Does one analysis seem to be smoother than the other?  If so, what would cause this?


3 - What happens as you increase the number of points considered for the analysis?  Is this
    desirable?  Why or why not?

'''